### Notebook for the label transfer of healthy skin to infected skin using `scANVI`

- **Developed by:** Carlos Talavera-López Ph.D
- **Würzburg Institute for Systems Immunology & Julius-Maximilian-Universität Würzburg**
- v2300704

### Import required modules

In [ ]:
import scvi
import anndata
import warnings
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt

### Set up working environment

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

In [ ]:
warnings.simplefilter(action = 'ignore')
scvi.settings.seed = 1712
%config InlineBackend.print_figure_kwargs = {'facecolor' : "w"}
%config InlineBackend.figure_format = 'retina'

In [ ]:
arches_params = dict(
    use_layer_norm = "both",
    use_batch_norm = "none",
    encode_covariates = True,
    dropout_rate = 0.2,
    n_layers = 2,
)

### Read in Healthy data

In [ ]:
SCC0120_1_Skin = sc.read_h5ad('../data/SCC0120_1_Skin_scANVI_annot_ctl230702.h5ad')
SCC0120_1_Skin

In [ ]:
SCC0120_1_Skin.obs['condition'].cat.categories

In [ ]:
SCC0120_1_HS = SCC0120_1_Skin[SCC0120_1_Skin.obs['condition'].isin(['healthy'])]
SCC0120_1_HS

In [ ]:
SCC0120_1_HI = SCC0120_1_Skin[SCC0120_1_Skin.obs['condition'].isin(['infected'])]
SCC0120_1_HI

In [ ]:
SCC0120_1_HI.obs['seed_labels'] = 'Unknown'

In [ ]:
healthy_raw.obs['seed_labels'].value_counts()

In [ ]:
adata = SCC0120_1_HS.concatenate(SCC0120_1_HI, batch_key = 'skin_condition', batch_categories = ['SCC0120_1_health', 'SCC0120_1_infected'], join = 'inner')
adata

### Select HVGs

In [ ]:
adata_raw = adata.copy()
adata.layers['counts'] = adata.X.copy()

sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes = 7000,
    layer = "counts",
    batch_key = "donor",
    subset = True
)

adata

### Transfer of annotation with scANVI

In [ ]:
scvi.model.SCVI.setup_anndata(adata, batch_key = 'donor', labels_key = "seed_labels")

In [ ]:
scvi_model = scvi.model.SCVI(adata, n_latent = 50, n_layers = 3)

In [ ]:
scvi_model.train()

Epoch 10/400:   2%|▏         | 9/400 [00:25<18:58,  2.91s/it, loss=3.03e+03, v_num=1]

### Label transfer with `scANVI` 

In [ ]:
scanvi_model = scvi.model.SCANVI.from_scvi_model(scvi_model, 'Unknown')

In [ ]:
scanvi_model.train()

In [ ]:
adata.obs["C_scANVI"] = scanvi_model.predict(adata)

- Extract latent representation

In [ ]:
adata.obsm["X_scANVI"] = scanvi_model.get_latent_representation(adata)

- Visualise corrected dataset

In [ ]:
sc.pp.neighbors(adata, use_rep = "X_scANVI", n_neighbors = 50, metric = 'minkowski')
sc.tl.umap(adata, min_dist = 0.3, spread = 4, random_state = 1712)
sc.pl.umap(adata, frameon = False, color = ['seed_labels', 'donor', 'cell_states', 'condition', 'group', 'batch',], size = 1, legend_fontsize = 5, ncols = 3)

### Export annotated object

In [ ]:
SCC0120_1_skin_annotated = adata[adata.obs['group'].isin(['SCC0120_1_skin'])]
SCC0120_1_skin_annotated

In [ ]:
del(SCC0120_1_skin_annotated.obs['predicted_doublets'])

In [ ]:
SCC0120_1_skin_annotated.write('../data/SCC0120_1_Skin_scANVI_annot_ctl230702.h5ad')